# 1 Introducción

Se realizará el cálculo del índice de masa coporal (IMC) a partir de un dataset con los datos del peso y altura de una población con computación paralela utilizando PyOpenCL. 

El IMC es un indicador de la grasa corporal en muchos individuo y se considera como indicador del riesgo para la salud.

Este índice, es utilizado por los profesionales de la atención sanitaria para analizar problemas relacionado con la obesidad. El BMI (en inglés) se utiliza para fijar los riesgos para la salud de una persona asociados a obesidad y exceso de peso.

El cálculo del IMC se realiza según la fórmula del IMC usando el sistema métrico. Es el peso en kilogramos divido por la altura (estatura en metros) al cuadrado obteniendo la siguiente ecuación:

<center>$ IMC=Peso(kg)/Altura(m)^{2}$</center>

El objetivo de este ejercicio es calcular un índice forma paralela utilizando PyOpenCL. De esta forma, pueden ser aplicables a datasets de grandes tamaños que incluyan gran cantidad de información de la población, en este caso, se realizará el cálculo de la IMC de un dataset de poco tamaño demostrativo y ejemplificador que puede ser escalable y de gran utilidad en el estudio de la salud de la población.

---
# 2 Armado del ambiente
Se debe colocar la URL del dataset que contenga la información del peso y la altura de los ciudadanos. El formato es peso segudio de una coma "," seguido de la altura siendo ambos números de tipo float.

In [31]:
#@title # 2.1 Parámetros de ejecución
#@markdown ---
#@markdown ### Especifique la URL del dataset con los datos de peso y altura de la población:
url_dataset = "https://raw.githubusercontent.com/AlexSvad/EA2-SOA/master/HPC/Ejercicio%203/datos_ciudadanos.txt" #@param {type:"string"}
if url_dataset.find(".txt") == -1:
    raise Exception("El dataset debe ser de un formato válido. En este caso debe ser .txt.")

## 2.2 Instalar en el cuaderno el módulo CUDA de Python.

In [1]:
!pip install pycuda

## 2.3 Instalar en el cuaderno el módulo OpenCL de Python.

In [2]:
!pip install pyopencl

---
# 3 Desarrollo
Ejecución del algoritmo del cálculo del IMC

In [28]:
import pyopencl as cl  # Se importa la API de OpenCL GPU de Python
import numpy as np
import pandas as pd

#Se lee los datos de los ciudadanos, el dataset debe estar conformado por el peso en KG seguido de una , la altura en centrímetros : [KG],[Altura]
try:
  df = pd.read_csv(url_dataset, names=['Peso','Altura'])
except:
   raise Exception("Error. El dataset posee un formato incorrecto. Debe tener el formato de Peso,Altura siendo floats ambos números")

#Se selecciona la plataforma, en este caso la primera por ser un entorno virtualizado
plataforma = cl.get_platforms()[0]

#Se selecciona el dispositivo, en este caso el primero por ser un entorno virtualizado
device = plataforma.get_devices()[0]

#Se crea el contexto del dispositivo (De la GPU virtual)
contexto = cl.Context([device])

#Se crea una cola de comandos a partir del contexto creado
cola = cl.CommandQueue(contexto) 

#Se crea un vector que contendrá el peso en KG
np_peso = np.copy(df[['Peso']]).astype(np.float32)

#Se crea un vector que contendrá la altura en CM
np_altura = np.copy(df[['Altura']]).astype(np.float32)

#Se crea un vector vacío que contendrá el resultado del IMC (índice de masa corporal)
np_imc = np.empty_like(np_peso)

#Se crea tres buffers correspondientes a las áreas de memoria del dispositivo GPU
cl_peso = cl.Buffer(contexto, cl.mem_flags.COPY_HOST_PTR, hostbuf=np_peso)
cl_altura = cl.Buffer(contexto, cl.mem_flags.COPY_HOST_PTR, hostbuf=np_altura)
cl_imc = cl.Buffer(contexto, cl.mem_flags.WRITE_ONLY, np_imc.nbytes)

#Se define el kernel con la función que calculará el IMC en forma paralela
kernel = \
"""
__kernel void calculo_imc(__global float* p, __global float* a, __global float* imc)
{
    int i = get_global_id(0);
    imc[i] = p[i] / ((a[i]/100)*(a[i]/100)); // El IMC del ciudadano se calcula: KG/Altura²
}
"""

#Se compila el código del kernel en OpenCL
programa = cl.Program(contexto, kernel).build()
programa.calculo_imc(cola, np_peso.shape, None, cl_peso, cl_altura, cl_imc)

#Se encola el programa para su ejecución, haciendo que los datos se copien al dispositivo
# - cola: la cola de comandos a la que se enviará el programa
# - np_peso.shape: una tupla de las dimensiones de los arrays
# - cl_peso, cl_altura, cl_imc: los espacios de memoria correspondiente a los arrays de peso, altura y los resultados del IMC

cola.finish()
np_arrays = [np_peso, np_altura, np_imc]
cl_arrays = [cl_peso, cl_altura, cl_imc]

#Se copia los resultados del IMC al host
for x in range(3):
    cl.enqueue_copy(cola, np_arrays[x], cl_arrays[x])

#Se guarda los resultados en un array
resultado_arr = {"Pesos de los ciudadanos":np_peso, "Alturas de los ciudadanos":np_altura, "Cálculo IMC":np_imc}

#Se imprime los resultados
for k in resultado_arr:
    print ("\n" + k + ":\n\n", resultado_arr[k])


Pesos de los ciudadanos:

 [[ 68. ]
 [ 66.5]
 [ 78. ]
 [ 58.1]
 [ 60.4]
 [ 60.9]
 [105.9]
 [ 90.6]
 [ 72.5]
 [ 80.6]
 [ 68.8]
 [ 82.4]
 [ 97.5]
 [ 70.8]
 [ 49.8]
 [ 56.4]
 [ 84.7]
 [ 54. ]
 [ 72.4]
 [ 85.1]
 [ 55.1]
 [ 84.8]
 [ 56.7]
 [ 51.8]
 [ 79. ]
 [ 59.8]
 [ 49.1]
 [ 63.2]
 [ 86.7]
 [ 49.1]
 [ 77.7]
 [ 83.5]
 [ 92.2]
 [ 61.1]
 [ 60.7]
 [ 85.8]
 [ 69.7]
 [ 55. ]
 [ 95.9]
 [ 80.6]]

Alturas de los ciudadanos:

 [[165. ]
 [156.8]
 [185. ]
 [140. ]
 [133.5]
 [154. ]
 [196. ]
 [179. ]
 [167. ]
 [178.9]
 [144. ]
 [164. ]
 [185. ]
 [174. ]
 [138. ]
 [147. ]
 [185. ]
 [157.5]
 [176. ]
 [187. ]
 [147. ]
 [186.1]
 [157.1]
 [136. ]
 [175. ]
 [156.9]
 [148.1]
 [164. ]
 [185.6]
 [149. ]
 [175.9]
 [188.8]
 [195.5]
 [165. ]
 [163.3]
 [181. ]
 [167. ]
 [152.2]
 [186.5]
 [171.1]]

Cálculo IMC:

 [[24.977045]
 [27.047651]
 [22.790361]
 [29.642857]
 [33.89023 ]
 [25.678867]
 [27.566643]
 [28.276272]
 [25.995913]
 [25.183401]
 [33.179016]
 [30.636528]
 [28.487951]
 [23.38486 ]
 [26.149967]
 [26.1002

---
# 4 Tabla de pasos


 Procesador | Función | Detalle
------------|---------|----------
CPU      |  url_dataset       | Lectura de la direcciún URL del dataset (txt) a procesar.
CPU      | !pip install pycuda    | Instala en el cuaderno los driver de CUDA para Python.
CPU      | !pip install pyopencl    | Instala en el cuaderno los drivers de OpenCL de Python.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  read_csv()          | Lee el dataset sugún la URL contenida en url_dataset.
CPU      |  names=['Peso','Altura']         | Se definen las columnas del dataset.
**GPU**      |  get_platforms()          | Obtiene las plataformas del entorno.
**GPU**      |  get_devices()          | Obtiene las dispositivos del entorno.
**GPU**     |  Context([device])          | Crea el contexto del dispositivo (De la GPU virtual).
CPU      |  CommandQueue(contexto)           | Crea una cola de comandos a partir del contexto creado.
CPU      |  df[[]]           | Procesa según la columna del dataset leído.
CPU      |  np.copy() | Copia y asigna el array enviado en como parámetro.
CPU      |  np.astype() | Define el tipo del array.
CPU      |  np.empty_like() | Genera el array destino vacío con el tamaño del array enviado como parámetro.
**GPU**     |  cl.Buffer() | Genera buffers correspondientes a las áreas de memoria del dispositivo GPU.
**GPU**  |  cl.Program().build()     | Compila el código del kernel en OpenCL.
**GPU**  |  programa.calculo_imc()    | Se define la función del cálculo del IMC del kernel y se encola el programa para su ejecución.
**GPU**  |   cl.enqueue_copy()    | Se copian los datos al dispositivo.
CPU      | print()   | Imprime los resultados.




---
# 5 Conclusiones

En conclusión, es posible procesar datasets con grandes volúmenes de datos con el motivo de realizar índices y estudios sobre una población utilizando procesamiento de forma paralela para lograr una mayor eficiencia a diferencia de la forma secuencial. Este ejercicio demostrativo realizado en PyOpenCL es un pequeño ejemplo del gran potencial que ofrece la computación paralela en el estudio de salud de la población al momento de procesar millones de registros de un país o varios países para realizar cálculos estádisticos, analizar la salud de los habitantes, realizar comparaciones y estudios de índices poblacionales. 

---
# 6 Bibliografía

[1] Estudio IMC Argentina: [PDF](https://www.sap.org.ar/docs/institucional/Del%20Pino.pdf)

[2] Importancia del IMC: [Página BMI](https://www.news-medical.net/health/What-is-Body-Mass-Index-(BMI)-(Spanish).aspx) 

[3] Fórmula del IMC: [Página healthy](https://www.healthychildren.org/Spanish/health-issues/conditions/obesity/Paginas/body-mass-index-formula.aspx) 

[4] Datasets de Argentina: [Página Oficial](https://datos.gob.ar/dataset)
